In [1]:
from fairseq.models.bart import BARTModel

2021-10-07 12:02:47.346480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-07 12:02:47.346663: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# # MBart Test
# mbart = BARTModel.from_pretrained('mbart.cc25.v2', checkpoint_file='model.pt')
# mbart.eval()

In [4]:
plbart = BARTModel.from_pretrained('PLBART/plbart_orig_pretrained_ckpt', checkpoint_file='model.pt')
plbart.eval()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(50005, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDropout()
     

In [5]:
fs_model = plbart.model

In [6]:
from transformers import PLBartConfig, PLBartForConditionalGeneration

In [7]:
hf_model = PLBartForConditionalGeneration.from_pretrained('plbart-base')

## Inputs

In [8]:
import sentencepiece as spm

In [9]:
vocab_filepath = "./PLBART/plbart_orig_pretrained_ckpt/sentencepiece.bpe.model"
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(vocab_filepath)
tokenizer.SetEncodeExtraOptions("")

True

In [10]:
texts = ["This is a sample text", "Another example here"]
text_0_tokens = tokenizer.EncodeAsIds(texts[0].strip())
text_1_tokens = tokenizer.EncodeAsIds(texts[1].strip())
# Need to check how is the original tokenizer

In [11]:
text_1_tokens += [1]*2

In [12]:
import numpy as np
import torch
import torch.nn as nn

In [13]:
input_ids = torch.from_numpy(np.array([text_0_tokens, text_1_tokens]))
attention_mask = torch.ones_like(input_ids)
attention_mask[1, -2:] = 0
token_type_ids = torch.zeros_like(input_ids)

# Encoder Embeddings

## Fairseq

In [14]:
fs_embeds, embed = fs_model.encoder.forward_embedding(input_ids)

In [15]:
# fs_model.encoder.quant_noise # None
# fs_model.encoder.layernorm_embedding #  not None
# fs_model.encoder.embed_positions # not None

## HuggingFace

In [16]:
hf_model.model.encoder.training

False

In [17]:
input_shape = input_ids.size()
inputs_embeds = hf_model.model.encoder.embed_tokens(input_ids) * hf_model.model.encoder.embed_scale
embed_pos = hf_model.model.encoder.embed_positions(input_shape)
hf_embeds = inputs_embeds + embed_pos
hf_embeds = hf_model.model.encoder.layernorm_embedding(hf_embeds)
hf_embeds = nn.functional.dropout(hf_embeds, p= hf_model.model.encoder.dropout, training= hf_model.model.encoder.training)


In [18]:
torch.allclose(hf_embeds[0], fs_embeds[0], atol=1e-5)

True

In [19]:
torch.allclose(hf_embeds[1,:-2], fs_embeds[1, :-2], atol=1e-5)

True

In [20]:
hf_embeds[1, -2:]

tensor([[-0.8719,  0.3062,  1.0597,  ..., -0.0096,  0.4988, -1.4163],
        [-0.8947,  0.2729,  0.8975,  ...,  0.3601,  0.6190, -1.4200]],
       grad_fn=<SliceBackward>)

In [21]:
fs_embeds[1, -2:] # Fairseq handles padding tokens differently, so they won't match.

tensor([[-0.6187,  0.1517,  1.1726,  ...,  0.1968,  0.5475, -1.0572],
        [-0.6187,  0.1517,  1.1726,  ...,  0.1968,  0.5475, -1.0572]],
       grad_fn=<SliceBackward>)

In [22]:
torch.allclose(hf_embeds[1,-2:], fs_embeds[1, -2:], atol=1e-5)

False

# Encoder Model

## Fairseq

In [23]:
fs_encoder = fs_model.encoder

In [24]:
src_lengths = torch.tensor([len(text_0_tokens), len(text_1_tokens)])

In [36]:
fs_encoder_out = fs_encoder(input_ids, src_lengths)

In [37]:
fs_encoder_out_encoder_out = fs_encoder_out.encoder_out.permute(1, 0, 2)

In [38]:
fs_encoder_out_encoder_out.shape

torch.Size([2, 5, 768])

## HuggingFace

In [39]:
hf_encoder = hf_model.model.encoder

In [40]:
hf_encoder_out = hf_encoder(input_ids, attention_mask)

In [41]:
hf_encoder_out = hf_encoder_out.last_hidden_state

In [43]:
torch.allclose(hf_encoder_out[0], fs_encoder_out_encoder_out[0], atol=1e-5)

True

In [44]:
torch.allclose(hf_encoder_out[1,:-2], fs_encoder_out_encoder_out[1,:-2], atol=1e-5)

True

# Decoder

## Fairseq

In [33]:
fs_decoder = fs_model.decoder

In [66]:
fs_decoder_out = fs_decoder(input_ids, fs_encoder_out)

In [67]:
len(fs_decoder_out)
fs_decoder_out_decoder_out = fs_decoder_out[0]

In [68]:
fs_decoder_out[1].keys()

dict_keys(['attn', 'inner_states'])

In [77]:
#  fs_decoder_out[1]['inner_states'][-1].permute(1, 0, 2).shape

In [78]:
fs_decoder_out_inner_state = fs_decoder_out[1]['inner_states'][-1].permute(1, 0, 2)

In [55]:
fs_decoder_out_decoder_out.shape

torch.Size([2, 5, 50005])

## HuggingFace

In [56]:
hf_decoder = hf_model.model.decoder

In [61]:
hf_decoder_out = hf_decoder(input_ids, attention_mask=attention_mask, encoder_hidden_states=hf_encoder_out, encoder_attention_mask=attention_mask)

In [64]:
hf_decoder_out.last_hidden_state.shape

torch.Size([2, 5, 768])

In [79]:
torch.allclose(hf_decoder_out.last_hidden_state[0], fs_decoder_out_inner_state[0], atol=1e-5)

True

In [80]:
torch.allclose(hf_decoder_out.last_hidden_state[1, :-2], fs_decoder_out_inner_state[1, :-2], atol=1e-5)

True

In [81]:
lm_logits = hf_model.lm_head(hf_decoder_out.last_hidden_state) + hf_model.final_logits_bias

In [82]:
lm_logits.shape

torch.Size([2, 5, 50005])

In [83]:
torch.allclose(lm_logits[0], fs_decoder_out_decoder_out[0], atol=1e-5)

True

In [84]:
torch.allclose(lm_logits[1, :-2], fs_decoder_out_decoder_out[1, :-2], atol=1e-5)

True